In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from typing import List, Union, Tuple, Optional, Dict, Callable

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "demo2", 'aviris' )
dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
dm.modal.data_dir = "/adapt/nobackup/projects/ilab/data/Aviris"
dm.modal.image_names = [ "ang20170720t004130_corr_v2p9" ]
TileManager.block_index = [1,1]
dm.proc_type = "gpu"

Opening log file:  '/home/tpmaxwel/.spectraclass/logging/aviris/demo2.log'
Using config file: '/gpfsm/ccds01/nobackup/projects/ilab/projects/spectraclass/defaults/config.py'
Using config file: '/home/tpmaxwel/.spectraclass/config/aviris/demo2.py'


/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


Here we define a set of class names and associated colors to be used in the classification process.

In [3]:
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

Here we start up the Spectraclass GUI.

In [4]:
dm.loadCurrentProject()
lm().setLabels( classes )
controller: SpectraclassController = app()
controller.gui()

Completed Reading raster file /adapt/nobackup/projects/ilab/data/Aviris/ang20170720t004130_corr_v2p9.tif, dims = ('band', 'y', 'x'), shape = (425, 7753, 9664), time=0.11 sec
Initializing GUI using controller <class 'spectraclass.gui.spatial.application.Spectraclass'>
Completed Reading raster file /adapt/nobackup/projects/ilab/data/Aviris/ang20170720t004130_corr_v2p9.tif, dims = ('band', 'y', 'x'), shape = (425, 7753, 9664), time=0.04 sec
Loading metadata from file: /adapt/nobackup/projects/ilab/data/Aviris/ang20170720t004130_corr_v2p9.mdata.txt


2022-02-10 15:22:42.871279: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-02-10 15:22:57.588362: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-10 15:22:57.588623: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-10 15:22:57.636115: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:61:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-02-10 15:22:57.636167: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-02-10 15:22:57.636220: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2

NameError: name 'Tuple' is not defined